#  CTI origin-location 
Calculate Covariance to the next line (//)  or next pixel (serial) in slices (along line and column)  of image difference  
to look for covariance variation over line or column , to identify if CTI effects are in a specific location in the device .
This is also useful to identify trap ( = negative correlation at low flux ) 

In [1]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
import numpy as np
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath


# Firefly client imports
from IPython.display import IFrame

In [2]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

### Set up the config for the ISR task.  This essentially turns off all processing other than overscan and bias correction.

In [3]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False
isr_config.doSaturationInterpolation=False

### Construct the `IsrTask` with the above configuration

In [4]:
isr = IsrTask(config=isr_config)

In [5]:
BOOTCAMP_REPO_DIR = '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'SFLAT_500'})

In [6]:
# Handle non-even series of images
# Tried to do something more clever, but not fully worked through yet
#etimes = []
#for visit in visits:
#    dId = {'visit': visit, 'detector': 2}
#    raw = butler.get('raw', **dId)
#    etimes.append(raw.getInfo().getVisitInfo().getExposureTime())
#exposure_times = numpy.array(etimes)
#unique_times, unique_indicies = numpy.unique(exposure_times, return_index=True)
#print(unique_indicies)
#indicies = unique_indicies.tolist() + [exposure_times.size - 1]

# Brute force for RTM-007, but inaccurate for all cases
bad_visits = []
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]):
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        print("Mismatched exptimes")
        bad_visits.append(visit1)

print(len(visits))
for bad_visit in bad_visits:
    visits.remove(bad_visit)
print(len(visits))

Mismatched exptimes
35
34


In [10]:
### config to compute the varaicne and covaraince 
#number of pair
nb_pair=int(len(visits)/2)
print('We saw %d pair of images '% (nb_pair))
#
# Cliping in number of sigma for the variance and mean
# default : not used 
#clip_sig=4.
# Size of the cov matrix 
#ncov_x=2
#ncov_y=2
ncov_x=2
ncov_y=2
# The cov matrix will be computed in nx*ny bin for each  amplifier , not just 1 cov matrix per amplifier ( amplifier is subdivided in nx*ny boxes)    
nx=2
ny=2 
# size of the ccd   ==> we should make this agnostic 
# x : column , y : line , first & last+1 
# in x exclusion region , and box size : should be function of the amplifier used ... we do for the worse case : side device
xoff=25
# first column to use
xf=0 + xoff   
# last column+1 to use 
xl=512 - xoff   
# y exclusion region , and box size 
yoff=10
#first line to use 
yf=0 + yoff     
# last line + 1 to use 
yl=2002  - yoff
# step size
ystep=int((yl-yf)/ny)
xstep=int((xl-xf)/nx)
#
# Temporary strorage before averaging over all pair of a given flux
temp_var=np.zeros((16,ny,nx,ncov_y,ncov_x, nb_pair))
temp_evar=np.zeros((16,ny,nx,ncov_y,ncov_x, nb_pair))
temp_nentry=np.zeros((16,ny,nx,ncov_y,ncov_x, nb_pair))
temp_mean=np.zeros((16,ny,nx, nb_pair))
temp_time=np.zeros(( nb_pair))



We saw 12 pair of images 


In [11]:
gain = {}
exp_time = {}

visits = visits[0:24]

i = 1
# counter of the number of pari seen
npair=0
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
    # Get ISR data for first image
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    bias1 = butler.get('bias', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    bias2 = butler.get('bias', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        print("Mismatched exptimes")
        continue
    # save the exposure time for this pair     
    temp_time[npair]=time1
    # run ISR on both images
    result1 = isr.run(raw1, bias=bias1)
    result2 = isr.run(raw2, bias=bias2)
    
    detector = result1.exposure.getDetector()
    #   channel id , for the time beeing put 3
    k=3
    amp = detector[k]
    #
    sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
    print ('first image of the pair  ready')
    #arr1 = sub_im1.getImage().getArray()
    sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
    print ('second image of the pair  ready')
    #arr2 = sub_im2.getImage().getArray()
    #print(sub_im1.getImage().getArray().shape)
    
    stats_im1 = afwMath.makeStatistics(sub_im1, afwMath.MEDIAN)
    stats_im2 = afwMath.makeStatistics(sub_im2, afwMath.MEDIAN)
    
    avg_flux1 = stats_im1.getValue(afwMath.MEDIAN)
    avg_flux2 = stats_im2.getValue(afwMath.MEDIAN)
    print('Flux level of the 2 images =',avg_flux1, avg_flux2)
    
    avg_flux = (avg_flux1 + avg_flux2) / 2
    
    sub_im1 /= avg_flux1
    sub_im2 /= avg_flux2
    
    # image difference on re-normed flux 
    diff_im = sub_im1.clone()
    diff_im -= sub_im2
    
    diff_im *= avg_flux
    #
    diff_array=diff_im.getImage().clone().getArray()
    
    # image average 
    avg_im  =   sub_im1.clone()
    avg_im  +=  sub_im2
    
    avg_im  *=  avg_flux 
    #
    avg_array=avg_im.getImage().clone().getArray()
    # compute the var and covariance within image boxes
    ix=0
    iy=0
    # loop on the different sub-boxes of a channel
    for y in range(yf,yl,ystep) :
        for x in range(xf,xl,xstep) :
            # for the current box the image diff
            # we should take a truncted mean ... but we will take a median for the moment 
            temp_mean[k,iy,ix,npair]=np.median(avg_array[y:y+ystep,x:x+xstep])
            #
            wd=diff_array[y:y+ystep,x:x+xstep]
            # compute the various variance and corvariance 
            for i in range(ncov_y) : 
                for j in range (ncov_x) : 
                    v1=wd[i:ystep,j:xstep]*wd[0:ystep-i,0:xstep-j]
                    if i != 0 and j != 0 :
                        # the initial version was a masked array - clipped to 4 sigma ... for the moment put it to a fix value
                        # n1=v1.count()
                        n1=ystep*xstep
                        v2=wd[i:ystep,0:xstep-j]*wd[0:ystep-i,j:xstep]
                        #n2=v2.count()
                        n2=ystep*xstep
                        temp_nentry[k,iy,ix,i,j,npair]=n1+n2
                        #weighted average of the mean in function of the statistic in each subf sample 
                        temp_var[k,iy,ix,i,j,npair]=(v1.mean()*n1+v2.mean()*n2)/temp_nentry[k,iy,ix,i,j,npair]
                        temp_evar[k,iy,ix,i,j,npair]=(v1.std()*n1+v2.std()*n2)/temp_nentry[k,iy,ix,i,j,npair]
                    else:
                        # the initial version was a masked array - clipped to 4 sigma ... for the moment put it to a fix value
                        #temp_nentry[k,iy,ix,i,j,npair]=v1.count()
                        temp_nentry[k,iy,ix,i,j,npair]=ystep*xstep
                        temp_var[k,iy,ix,i,j,npair]=v1.mean()
                        temp_evar[k,iy,ix,i,j,npair]=v1.std()
            #diff_std=wd.std()
            #diff_var=diff_std**2
            ix+=1
        #
        ix=0
        iy+=1
    # move to next pair of image
    npair+=1

first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1050.6278076171875 1061.7132568359375
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1065.46337890625 1060.052490234375
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1050.593994140625 1052.67822265625
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1055.298095703125 1071.1209716796875
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1063.03173828125 1069.314208984375
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1071.1331787109375 1069.8699340820312
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1057.985107421875 1066.541015625
first image of the pair  ready
second image of the pair  ready
Flux level of the 2 images = 1074.68798828

In [15]:
# average over the pairs  of same exposure time 
itime=np.argsort(temp_time)
t_ref=temp_time[itime[0]]
selected_pair=[itime[0]]
results_final=[]
for ipair in range(1,npair) :
    if temp_time[itime[ipair]]>t_ref :
        results_final.append({'mean':temp_mean[:,:,:,selected_pair].mean(axis=3),
                              'var':temp_var[:,:,:,:,:,selected_pair].mean(axis=5),
                              'evar':temp_evar[:,:,:,:,:,selected_pair].mean(axis=5),
                              'nentry':temp_nentry[:,:,:,:,:,selected_pair].sum(axis=5),
                              'npair':len(selected_pair),
                              'time':temp_time[selected_pair].mean()})
        selected_pair=[]
    #
    selected_pair.append(itime[ipair])
# average the last serie of pair 
results_final.append({'mean':temp_mean[:,:,:,selected_pair].mean(axis=3),
                              'var':temp_var[:,:,:,:,:,selected_pair].mean(axis=5),
                              'evar':temp_evar[:,:,:,:,:,selected_pair].mean(axis=5),
                              'nentry':temp_nentry[:,:,:,:,:,selected_pair].sum(axis=5),
                              'npair':len(selected_pair),
                              'time':temp_time[selected_pair].mean()})
        

In [21]:
print('surprise ! we get quite different covariance in the # image parts !!!!')
print('cov(1,0) for the 4 image parts')
print(results_final[0]['var'][k,:,:,0,1])
print('we got comparable variance ???')
print(results_final[0]['var'][k,:,:,0,0])
print('and comparable flux ???')
print(results_final[0]['mean'][k,:,:])


surprise ! we get quite different covariance in the # image parts !!!!
cov(1,0) for the 4 image parts
[[ 6.0703873   0.67166394]
 [ 1.86618502 17.88805332]]
we got comparable variance ???
[[3234.71791585 3243.89805094]
 [3239.93607585 3262.68546549]]
and comparable flux ???
[[2127.58412679 2128.45277913]
 [2126.80556234 2127.50295003]]
